In [3]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


In [2]:
import numpy as np
import cv2 
import os


In [4]:
def dist(x1,x2):
      return np.sqrt(sum((x1-x2)**2))

def knn(X,Y,k=10):
  vals=[]
  m=X.shape[0]

  for i in range(m):
    ix=X[i, :-1]
    iy=X[i,-1]
    d=dist(Y,ix)
    vals.append((d,iy))

  dk=sorted(vals,key=lambda x:x[0])[:k]
  labels=np.array(dk)[:,-1]
  output=np.unique(labels,return_counts=True)
  index=np.argmax(output[1])
  return output[0][index]

In [ ]:
#Init Camera

cap=cv2.VideoCapture(0)

#Face Detection
face_cascade=cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")
skip=0
dataset_path='/Users/harshkashyap/'

face_data=[]
label=[]

class_id=0 #Labels for given file
names = {} #Maoping between id name

#Data Preperation
for fx in os.listdir(dataset_path):
    if fx.endswith('.npy'):
        #creating a map id between class id and name
        names[class_id]=fx[:-4]  #Harsh without .npy part
        data_item = np.load(dataset_path+fx)
        face_data.append(data_item)
        
        
        #Create labels for class
        target= class_id*np.ones((data_item.shape[0],))
        class_id +=1
        label.append(target)
    
face_dataset = np.concatenate(face_data,axis=0)
face_labels = np.concatenate(label,axis=0).reshape((-1,1))

print(face_dataset.shape)
print(face_labels.shape)

trainset=np.concatenate((face_dataset,face_labels),axis=1)
print(trainset.shape)
        
        
        
        
        
#testing part
while True:
    ret,frame= cap.read()
    if ret==False:
        continue
    faces=face_cascade.detectMultiScale(frame,1.3,5)
    
    for face in faces:
        x,y,w,h= face
        
        #get the face ROI
        offset=10
        face_section=frame[y-offset:y+h+offset,x-offset:x+w+offset]
        face_section=cv2.resize(face_section,(100,100))
        
        #Predicted output
        out=knn(trainset,face_section.flatten())
        
        #display on the screen name and rectange around it 
        pred_name=names[int(out)]
        cv2.putText(frame,pred_name,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2,cv2.LINE_AA)
        cv2.rectangle(frame,(x,y),(x+h,y+w),(0,255,255),2)
    
    cv2.imshow("Faces ",frame)
    
    key= cv2.waitKey(1) & 0xFF
    if (key == ord('q')):
        break
    
cap.release()
for i in range (1,3):
    cv2.destroyAllWindows()
    cv2.waitKey(1)

(26, 30000)
(26, 1)
(26, 30001)
